In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
plt.ylim(0, 100)
import ipywidgets as ipy
import seaborn as sns
import plotly
import plotly.express as px
import plotly.io as pio
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import bqplot
import numpy as np
import ipywidgets as ipy
from ipywidgets import interact
#from bqplot import pyplot as plt
from bqplot import LinearScale, Lines, Axis, Figure, DateScale, ColorScale

In [ ]:
df = pd.read_csv("Finance_data.csv")

In [ ]:
df.head()

In [ ]:
data = df[['gender','age','Mutual_Funds', 'Equity_Market', 'Government_Bonds', 'Fixed_Deposits']]


In [ ]:
data.head()

In [ ]:
df_age = df.groupby('age', as_index = False).agg({"Mutual_Funds":"sum","Equity_Market":"sum","Government_Bonds":"sum", "Fixed_Deposits":"sum"})
df_age

In [ ]:
df_gender = df.groupby('gender', as_index = False).agg({"Mutual_Funds":"sum","Equity_Market":"sum","Government_Bonds":"sum", "Fixed_Deposits":"sum"})
df_gender

In [ ]:
total_mutual= df.groupby("age")["Mutual_Funds"].sum() #finding total square footage based on year acquired

In [ ]:
float(max(total_mutual.values)), float(min(total_mutual.values))

In [ ]:
x = bqplot.LinearScale(max=max(df['age']), min=min(df['age']))

In [ ]:
y = bqplot.LinearScale(max = float(max(total_mutual.values)),min=float(min(total_mutual.values)))


In [ ]:
x_age  = bqplot.Axis(scale = x, label = 'Age')
y_mutual  = bqplot.Axis(scale = y, label = 'Mutual_Funds', orientation = 'vertical')


pan = bqplot.interacts.PanZoom(scales={'x':[x],'y':[y]})


finance_bar = bqplot.Bars(x=total_mutual.index, y=total_mutual.values, scales={'x':x, 'y':y})

figb = bqplot.Figure(marks = [finance_bar], axes = [x_age,y_mutual],interaction=pan)

In [ ]:
figb

In [ ]:
pivot_table_finance = pd.pivot_table(df, values='Mutual_Funds', index='gender',
                    columns='age', aggfunc=np.sum)
#Finding the total Mutual Funds for a unique set of age and gender

In [ ]:
pivot_table_finance

In [ ]:
pivot_table_finance.fillna(0)

In [ ]:
pivot_table_finance.columns

In [ ]:
#Grid Heatmap

#setting the color scale, x and y axis scales
col1 = bqplot.ColorScale(scheme='Red')
x1 = bqplot.OrdinalScale()
y1 = bqplot.OrdinalScale()

#Axes
col_ax = bqplot.ColorAxis(scale=col1, orientation='vertical', side='right')
x_ax = bqplot.Axis(scale=x1)
y_ax = bqplot.Axis(scale=y1, orientation='vertical')

#Mark (heatmap)
heat_map = bqplot.GridHeatMap( color = pivot_table_finance,row= pivot_table_finance.index,
                              column = pivot_table_finance.columns,
                              scales={'color':col1, 'row':y1, 'column':x1},
                              interactions={'click':'select'},
                              selected_style={'fill':'blue'})


# plotting the heatmap
fig = bqplot.Figure(marks=[heat_map],axes=[col_ax, x_ax, y_ax])
fig

In [ ]:
myLabel=ipy.Label() #defining a label element

## Visualization 1

In [ ]:
#Grid Heatmap

col1 = bqplot.ColorScale(scheme='Red')
x1 = bqplot.OrdinalScale()
y1 = bqplot.OrdinalScale()

#Axes
col_ax = bqplot.ColorAxis(scale=col1, orientation='vertical', side='right')
x_ax = bqplot.Axis(scale=x1, label = 'gender')
y_ax = bqplot.Axis(scale=y1, label = 'age', orientation='vertical')

#Mark (heatmap)
heat_map = bqplot.GridHeatMap(color= pivot_table_finance, row= pivot_table_finance.index,
                              column = pivot_table_finance.columns,
                              scales={'color':col1, 'row':y1, 'column':x1},
                              interactions={'click':'select'},
                              selected_style={'fill':'blue'})

#Function 
def on_selection(change):
    if len(change['owner'].selected==1):
        i,j=change['owner'].selected[0]
        v=pivot_table_finance.iloc[i,j].mean()
        myLabel.value='Mean of Total Mutual Funds ='+str(v)
        
        #linking bar plot and grid heatmap together
        f1 = pivot_table_finance[pivot_table_finance.index.str.contains(pivot_table_finance.index[i])]
        fm = df[df['gender'] == f1.index.values[0]]
        total_mutual_fund= fm.groupby("age")["Mutual_Funds"].sum()
        finance_bar.x = total_mutual_fund.index
        finance_bar.y = total_mutual_fund.values



#Calling the function created for interaction
heat_map.observe(on_selection,'selected')

# plotting grid heatmap
fig = bqplot.Figure(marks=[heat_map],axes=[col_ax, x_ax, y_ax])

#Dashboard (with grid heatmap on left side and bar plot on right side)
figures = ipy.HBox([fig,figb])
fig.layout.min_width='500px'  #figure size of grid heatmap
figb.layout.min_width='500px' #figure size of bar plot
myDashboard = ipy.VBox([myLabel,figures])
myDashboard


## Visualization 2

In [ ]:
df_gender = df.groupby('gender', as_index = False).agg({"Mutual_Funds":"sum","Equity_Market":"sum","Government_Bonds":"sum", "Fixed_Deposits":"sum"})

In [ ]:
def myfunction(gender):
    plt.rcParams["figure.figsize"] = [20.00, 10.00]
    plt.rcParams["figure.autolayout"] = True
    f, axes = plt.subplots(2,2, figsize= (15,15))

    fig1 = sns.barplot(x="gender", y="Equity_Market", data = df_gender[df_gender['gender']==gender], ax=axes[0,0], palette='rocket')
    fig1.set_ylim(0,100)
    fig2 = sns.barplot(x="gender", y="Mutual_Funds", data = df_gender[df_gender['gender']==gender], ax=axes[0,1], palette='Oranges_r')
    fig2.set_ylim(0,100)
    fig3 = sns.barplot(x="gender", y="Government_Bonds", data = df_gender[df_gender['gender']==gender], ax=axes[1,0], palette='BuGn')
    fig3.set_ylim(0,100)
    fig4 = sns.barplot(x="gender", y="Fixed_Deposits", data = df_gender[df_gender['gender']==gender], ax=axes[1,1], palette='gnuplot')
    fig4.set_ylim(0,100)

interact(myfunction, gender=['Male', 'Female']);

## Visualization 3

In [ ]:
df_age = df.groupby('age', as_index = False).agg({"Mutual_Funds":"sum","Equity_Market":"sum","Government_Bonds":"sum", "Fixed_Deposits":"sum"})

In [ ]:
def myfunction(age):
    plt.rcParams["figure.figsize"] = [20.00, 10.00]
    plt.rcParams["figure.autolayout"] = True
    f, axes = plt.subplots(2,2, figsize= (15,15))

    fig1 = sns.barplot(x="age", y="Equity_Market", data = df_age[df_age['age']==age], ax=axes[0,0], palette='Oranges_r')
    fig1.set_ylim(0,35)
    fig2 = sns.barplot(x="age", y="Mutual_Funds", data = df_age[df_age['age']==age], ax=axes[0,1], palette='rocket')
    fig2.set_ylim(0,35)
    fig3 = sns.barplot(x="age", y="Government_Bonds", data = df_age[df_age['age']==age], ax=axes[1,0], palette='gnuplot')
    fig3.set_ylim(0,35)
    fig4 = sns.barplot(x="age", y="Fixed_Deposits", data = df_age[df_age['age']==age], ax=axes[1,1], palette='BuGn')
    fig4.set_ylim(0,35)

interact(myfunction, age=df_age['age']);

## Writeup

Author: Reethika Renganathan

Project Title: Comparitive analysis of different Investment Avenues

Idea behind the project:
The main idea behind the project is to identify how and where do people invest their money. We all know it's very important to be aware of all the different investment options that are available to us, but do people actually know where their money can yeild more returns? This project tries to analyze that by creating different visualizations that help you understand how different parametrs like gender and age are plotted against the various investment avenues from a dataset that has entries from different sets of people who have answered a questionnaire on their financial investments. 

Steps to tackle the propblem: 
1.) At the very beginning, since the data collected from the questionnaire had to be refined in such way that all the columns contain values, and to make sure that there are no discrepencies in the data, I tried to clean the data by removing empty data values and making the data uniform.

2.) After cleaning the data to make it uniform, I tried to group the data according to age and then according to gender to find out if there was any relation between a person's age and how they invest. I followed the same procedure for grouping the data according to gender versus the various investment avenues that they invest in. 

3.) Not only did I group them according to age and gender, I also aggregated the values in the investment options by calculating the sum of each option for a set of value chosen. For eg: for all indivuals of the age 27, add all the values for 'Mutual Funds'.

4.) Then, I tried making a fun visualization as a bar plot between Mutual Funds and Age with an interesting interactive feature of 'PanZoom'. Through this visualization, you can see the different levels of investments by people of a certain age in Mutual Funds. With the interactive element, you can zoom in on the figure and see the calculations clearly. 

5.) For the next visualization, I tried creating a dashboard with two plots that interacted with each other. In the first plot, I've created a heatmap which is nothing but a style of representing data values by different colours. With the help of the tool tip on the side, you can identify which colour denotes which value. Usually the value goes from lower to higher. 

6.) The heatmap which has different colours, is basically showing the mean of the total mutual funds in each cell. So when you click on a particular cell, you will come to know the exact value of the mean of the total mutual fund for that particular cell. 

7.) Now with only this heatmap, it's a little difficult for a person to identify what it is trying to explain, so for that, I tried linking this to a bar plot with 'Mutual Funds' as y axis and 'Age' as x axis. 

8.) So by making it interactive, we can observe that for every cell in the heatmap, when you select the Mean of Total Mutual Funds, there is a bar plot with values of different ages and their investment in mutual funds. 

9.) The next two maps are again a way of analyzing how age and gender are related to investment in Mutual Funds, Government Bonds, Equity Market, and Fixed Deposit. 

10.) With the dropdown given at the top, when you select any value in it (gender for the first and age for the second), you will see how the four graphs are changing for a different set of value each time you select something. 



## References:
1. https://uiuc-ischool-dataviz.github.io/is445_spring2022/nbv.html?notebook_name=%2Fis445_spring2022%2Fweek07%2FinClass_week07.ipynb
2. https://uiuc-ischool-dataviz.github.io/is445_spring2022/nbv.html?notebook_name=%2Fis445_spring2022%2Fweek05%2FinClass_week05.ipynb


## Dataset:
1. https://www.kaggle.com/datasets/nitindatta/finance-data